# Example 1: Get protein information, run BLAST, and dump the results into a database


In [1]:
%reload_ext autoreload
%autoreload 2

import toml

from pyEED.core.proteinsequence import ProteinSequence
from pyEED.ncbi.utils import get_nucleotide_sequences

## Query NCBI

The pyEED library is centered around the `ProteinSequence` object, which integrates available information on protein sequence, corresponding nucleotide sequence, as well as regions and sites within the sequences. The `ProteinSequence` can be initialized directly with a protein sequence accession number.

In [2]:
aldolase = ProteinSequence.from_ncbi("NP_001287541.1")
print(aldolase)

ProteinSequence
├── id = NP_001287541.1
├── name = aldolase 1, isoform M
├── sequence = MTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVILFHETLYQKADDGTPFAEILKKKGIILGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPEEIALATVQALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLIRPWALTFSYGRALQASVLRAWAGKKENIAAGQNELLKRAKANGDAAQGKYVAGSAGAGSGSLFVANHAY
├── organism
│   └── Organism
│       ├── id = taxon:7227
│       ├── name = Drosophila melanogaster
│       └── taxonomy_id = 7227
├── regions
│   └── 0
│       └── Region
│           ├── id = region0
│           ├── start = 14
│           ├── end = 361
│           ├── note = Fructose-bisphosphate aldolase class-I; pfam00274
│           ├── name = Glycolytic
│           └── cross_reference = CDD:425574
├── sites
│   ├── 0
│   │   └── Site
│   │       ├── id = site0
│   │       ├── name = active
│   │       ├── type = active
│   │       ├── pos

In [3]:
# Get the corresponding nucleotide sequence
aldolase.get_nucleotide_seq()
print(aldolase.coding_sequence)

NucleotideSequence
├── id = NM_001300612.1
├── regions
│   └── 0
│       └── Region
│           ├── id = NM_001300612.1
│           ├── start = 278
│           └── end = 1363
├── molecule_type = mRNA
├── gene_id = Ald1
└── sequence = ATGACGACCTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTGCGCGAAATCGCCCAGAAAATCGTTGCCCCCGGCAAGGGAATCCTCGCCGCCGATGAGTCCGGCCCAACCATGGGCAAGCGTCTGCAGGACATCGGCGTGGAGAACACCGAGGACAACCGCCGTGCCTACCGTCAGCTGTTGTTCAGCACTGACCCCAAGCTGGCCGAGAACATCTCTGGAGTGATCCTGTTCCACGAGACCCTCTACCAGAAGGCCGATGATGGCACCCCCTTCGCCGAGATCCTGAAGAAGAAGGGAATCATTCTGGGCATCAAGGTCGACAAGGGTGTTGTCCCACTGTTCGGCTCTGAGGATGAGGTCACCACCCAGGGTCTGGATGACCTGGCCGCCCGTTGCGCCCAGTACAAGAAGGACGGTTGCGACTTCGCCAAGTGGCGTTGCGTCCTGAAGATCGGCAAGAACACCCCATCCTACCAGTCGATCCTGGAGAACGCCAATGTCCTGGCCCGCTACGCCTCCATCTGCCAGTCGCAGCGCATCGTCCCAATTGTGGAGCCCGAGGTTCTGCCCGATGGCGATCACGATCTGGACCGCGCCCAGAAGGTCACCGAGACCGTCCTGGCCGCCGTCTACAAGGCCCTGAGCGACCACCACGTCTACCTGGAGGGTACTCTGCTGAAGCCCAACATGGTCACCGCCGGTCAGTCGGCCAAGAAGAACACCCCCGAGGAGATCGCCCTGGCCACCGTGCAGGCTC

## BLAST search

In [4]:
blast_results = aldolase.pblast(n_hits=10)

Running pblast search for aldolase 1, isoform M from Drosophila melanogaster...


Fetching protein sequences: 100%|██████████| 10/10 [00:18<00:00,  1.85s/it]


In [5]:
get_nucleotide_sequences(blast_results)

Fetching nucleotide sequences: 100%|██████████| 10/10 [00:07<00:00,  1.33it/s]


## Storing `ProteinSequence`s in a database



In [6]:
from sdrdm_database import DBConnector

### Setting up a local MySQL database

First, a local MySQL database needs to be setup. Therefore, we run a docker container with a MySQL database. 

>[!NOTE]
>
>If docker is not isntalled on your system, please follow the instructions on the [docker website](https://docs.docker.com/get-docker/).


In case this notebook is run on a macOS system with a M1 chip, the following command needs to be run in the terminal first:

>```bash
>export DOCKER_DEFAULT_PLATFORM=linux/amd64
>```

Next, navigate to the directory where this notebook is located and run the following command to start the docker container:

>```bash
>docker compose up -d
>```

### Create tables for `ProteinSequence` and `NucleotideSequence`

In [7]:
!sdrdm-db --root-obj ProteinSequence --model-path https://github.com/PyEED/pyeed.git --env-file env.toml

🎉 Connected                                                                                        

🚀 Creating tables for data model ProteinSequence
│
├── Fetching markdown model from GitHub
├── Table __model_meta__ not existing. Adding to DB!
├── Added table model 'ProteinSequence' to __model_meta__ table
├── Model 'ProteinSequence' already registered. Skipping.
├── Created table 'ProteinSequence'
├── Added table model 'ProteinSequence_coding_sequence' to __model_meta__ table
├── Created table 'ProteinSequence_coding_sequence'
├── Added table model 'NucleotideSequence_regions' to __model_meta__ table
├── Created table 'NucleotideSequence_regions'
├── Added table model 'ProteinSequence_sites' to __model_meta__ table
├── Created table 'ProteinSequence_sites'
├── Created table 'Site_positions'
├── Added table model 'ProteinSequence_regions' to __model_meta__ table
├── Created table 'ProteinSequence_regions'
├── Added table model 'ProteinSequence_organism' to __model_meta__ table
├── Cre

In [8]:
# Establish a connection to the database
db = DBConnector(**toml.load(open("./env.toml")))

🎉 Connected                                                                                        


In [9]:
# Get an overview over the created tables
db.connection.tables

Tables
------
- NucleotideSequence_regions
- ProteinSequence
- ProteinSequence_coding_sequence
- ProteinSequence_organism
- ProteinSequence_regions
- ProteinSequence_sites
- Site_positions
- __model_meta__

### Populate the database with `ProteinSequence`s

In [10]:
# Insert all blast results into the database
db.insert(*blast_results, verbose=True)

Added dataset ProteinSequence (ce49f3d4-03a5-4943-a206-62bc225088f2)
Added dataset ProteinSequence (17c7fa21-18a6-4d61-918a-2ad441863bd8)
Added dataset ProteinSequence (68ac0244-7402-4c3a-a532-1e62e97f64de)
Added dataset ProteinSequence (4075bec0-4faa-4386-9572-64a47ee463dc)
Added dataset ProteinSequence (3254d412-9983-4ffe-abc6-4e1c93cd3047)
Added dataset ProteinSequence (56fd0978-0d54-4392-baa9-abe17856d280)
Added dataset ProteinSequence (158ad489-eef8-46fb-9a72-06eeef3f3034)
Added dataset ProteinSequence (24d03eb8-1645-491b-9278-0ce1c6c09fe2)
Added dataset ProteinSequence (292a1ef2-bc13-4488-aed5-d5ab364e8fe5)
Added dataset ProteinSequence (0eb2870a-460e-4940-8b75-8a30e58037e9)


### Look at entries in the database

In [11]:
db.connection.table("ProteinSequence")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name                                        ┃ sequence                                                                         ┃ ec_number ┃ mol_weight ┃ nr_id  ┃ uniprot_id ┃ pdb_id ┃ ProteinSequence_id                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string                                     │ !string                                                                          │ string    │ float64    │ string │ string     │ string │ !string                              │
├─────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────┼───────────┼────────────┼────────┼────────────┼────────┼──────────────────────────────────────┤
│ fructose-bisphosphate aldolase isoform X2   │ MTTYFNYPSKELQDELRDIAQRIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVIL… │ NULL      │    38930.0 │ NULL   │ NULL       │ NULL   │ 0eb2870a-460e-4940-8b75-8a30e58037e9 │
│ fructose-bisphosphate aldolase isoform X1   │ MTTYFNYPSKELQDELRDIAQRIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVIL… │ NULL      │    38916.0 │ NULL   │ NULL       │ NULL   │ 158ad489-eef8-46fb-9a72-06eeef3f3034 │
│ aldolase 1, isoform M                       │ MTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVIL… │ 4.1.2.13  │    38916.0 │ NULL   │ NULL       │ NULL   │ 17c7fa21-18a6-4d61-918a-2ad441863bd8 │
│ aldolase gamma                              │ MTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTHGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVIL… │ NULL      │        nan │ NULL   │ NULL       │ NULL   │ 24d03eb8-1645-491b-9278-0ce1c6c09fe2 │
│ fructose-bisphosphate aldolase isoform X2   │ MTTYFNYPSKELQDELRDIAQRIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVIL… │ NULL      │    38930.0 │ NULL   │ NULL       │ NULL   │ 292a1ef2-bc13-4488-aed5-d5ab364e8fe5 │
│ Chain A, FRUCTOSE 1,6-BISPHOSPHATE ALDOLASE │ XTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVIL… │ NULL      │        nan │ NULL   │ NULL       │ 1FBA_A │ 3254d412-9983-4ffe-abc6-4e1c93cd3047 │
│ fructose-bisphosphate aldolase isoform X1   │ MTTYFNYPSKELQDELRDIAQRIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVIL… │ NULL      │    38930.0 │ NULL   │ NULL       │ NULL   │ 4075bec0-4faa-4386-9572-64a47ee463dc │
│ fructose-bisphosphate aldolase isoform X3   │ MTTYFNYPSKELQDELRDIAQRIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVIL… │ NULL      │    38929.0 │ NULL   │ NULL       │ NULL   │ 56fd0978-0d54-4392-baa9-abe17856d280 │
│ fructose-bisphosphate aldolase isoform X1   │ MTTYFNYPSKELQDELREIAQRIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVIL… │ NULL      │    38944.0 │ NULL   │ NULL       │ NULL   │ 68ac0244-7402-4c3a-a532-1e62e97f64de │
│ FI14722p                                    │ MTLIASEGIIAEVSVTLLSYKNIIVENYTRISKMTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQD… │ NULL      │        nan │ NULL   │ NULL       │ NULL   │ ce49f3d4-03a5-4943-a206-62bc225088f2 │
└─────────────────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────────┴───────────┴────────────┴────────┴────────────┴────────┴──────────────────────────────────────┘

In [13]:
# Lets filter the blast results for a specific organism
target = "Drosophila melanogaster"

# First, join the ProteinSequence table with the ProteinSequence_organism table
prot_seqs = db.connection.table("ProteinSequence")
organisms = db.connection.table("ProteinSequence_organism")
joined = prot_seqs.join(
    organisms,
    prot_seqs.ProteinSequence_id == organisms.ProteinSequence_id,
    rname="organism_{name}",
)

# Next, filter the joined table for the target organism
filtered = joined.filter(joined.organism_name == target)
filtered

# Finally, we can get the corresponding ProteinSequence objects
results = db.get("ProteinSequence", filtered)
print(results[0])

ProteinSequence
├── id = 3254d412-9983-4ffe-abc6-4e1c93cd3047
├── name = Chain A, FRUCTOSE 1,6-BISPHOSPHATE ALDOLASE
├── sequence = XTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVILFHETLYQKADDGTPFAEILKKKGIILGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPEEIALATVQALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLIRPWALTFSYGRALQASVLRAWAGKKENIAAGQNELLKRAKANGDAAQGKYVAGSAGAGSGSLFVANHAY
├── organism
│   └── Organism
│       ├── id = 6ead49b7-28c7-4c45-b85b-967a7e31a307
│       ├── name = Drosophila melanogaster
│       └── taxonomy_id = 7227
├── regions
│   └── 0
│       └── Region
│           ├── id = ae562021-856a-47bc-a59c-65b151090404
│           ├── start = 14
│           ├── end = 361
│           ├── note = Fructose-bisphosphate aldolase class-I; pfam00274
│           ├── name = Glycolytic
│           └── cross_reference = CDD:425574
├── sites
│   ├── 0
│   │   └── Site
│   │  